<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 理解嵌入层和线性层之间的区别

- PyTorch中的嵌入层与执行矩阵乘法的线性层实现相同的功能；我们使用嵌入层的原因是计算效率
- 我们将通过PyTorch中的代码示例一步步查看这种关系

In [1]:
import torch

print("PyTorch version:", torch.__version__)

PyTorch version: 2.8.0+cu126


<br>
&nbsp;

## Using nn.Embedding

In [2]:
# 假设我们有以下三个训练样本，
# 它们可能表示LLM上下文中的token ID
idx = torch.tensor([2, 3, 1])

# 嵌入矩阵的行数可以通过获取最大token ID + 1来确定。
# 如果最高的token ID是3，那么我们需要4行，对应可能的
# token ID 0、1、2、3
num_idx = max(idx)+1

# 所需的嵌入维度是一个超参数
out_dim = 5

- 如果用简单的embedding layer:

In [3]:
# 我们使用随机种子来保证可重复性，因为
# 嵌入层中的权重是用小的随机值初始化的
torch.manual_seed(123)

embedding = torch.nn.Embedding(num_idx, out_dim)

我们可以快速地查看下

In [4]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

- 然后，我们可以使用嵌入层来获取ID为1的训练样本的向量表示：

In [5]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 可视化处理过的内容

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/1.png" width="400px">

- 类似地，我们可以使用嵌入层来获取ID为2的训练样本的向量表示：

In [6]:
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/2.png" width="400px">

- 现在，让我们转换之前定义的所有训练样本：

In [7]:
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 底层实现本质上仍然是相同的查找机制：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/3.png" width="450px">

<br>
&nbsp;

## Using nn.Linear

- 现在，我们将演示上面的嵌入层与PyTorch中`nn.Linear`层在one-hot编码表示上的效果完全相同。
- 首先，我们将token ID转换为独热编码表示：

In [8]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

- 接下来，我们初始化一个`Linear`层，它执行矩阵乘法$X W^\top$：

In [9]:
torch.manual_seed(123)
linear = torch.nn.Linear(num_idx, out_dim, bias=False)
linear.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

此代码片段初始化一个线性层并显示其权重：

*   `torch.manual_seed(123)`：设置 PyTorch 的随机种子。这确保了线性层权重的初始化是可重现的。
*   `linear = torch.nn.Linear(num_idx, out_dim, bias=False)`：创建一个线性层。
    *   `num_idx`：输入特征的数量，由最大 token ID + 1 确定。
    *   `out_dim`：输出特征的数量（期望的嵌入维度）。
    *   `bias=False`：表示该层不应学习偏差项。这样做是为了与嵌入层进行更直接的比较。
*   `linear.weight`：访问并显示线性层的权重矩阵。此矩阵使用小的随机值初始化。

- PyTorch中的线性层也是用小的随机权重进行初始化的
- 为了与上面的`Embedding`层进行直接对比，我们需要使用相同的小随机权重，这就是为什么我们在这里对他们重新赋值：

In [10]:
linear.weight = torch.nn.Parameter(embedding.weight.T)

此代码将 `embedding.weight.T`（来自 `nn.Embedding` 层的权重的转置）赋值给 `linear.weight`（来自 `nn.Linear` 层的权重）。`torch.nn.Parameter()` 用于将张量标记为模型的参数，以便在训练期间可以对其进行优化。

这样做是为了确保 `nn.Linear` 层使用与 `nn.Embedding` 层**转置后**完全相同的权重进行初始化。这是因为 `nn.Embedding` 层在内部执行查找操作，而 `nn.Linear` 层执行矩阵乘法 $XW^\top$。为了使这两者等价，`linear` 层的权重 $W$ 需要是 `embedding` 层权重的转置。这允许我们直接比较在独热编码输入上使用 `nn.Linear` 的结果与在原始 token ID 上使用 `nn.Embedding` 的结果，证明它们是等效的。

- 现在，我们可以在输入的one-hot编码表示上使用线性层：

In [11]:
linear(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

- 如我们所见，这与使用嵌入层时得到的结果完全相同：

In [12]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 实际上，底层对第一个训练样本的token ID进行的计算如下：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/4.png" width="450px">

- 对于第二个训练样本的token ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/5.png" width="450px">

- 由于每个one-hot编码行中只有一个索引为1，其余的索引都为0（这是设计的结果），因此这个矩阵乘法实际上与one-hot元素的查找相同。
- 在one-hot编码上使用矩阵乘法与嵌入层的查找等价，但当处理较大的嵌入矩阵时，这种做法可能效率较低，因为存在大量无意义的零乘法。